In [1]:
__author__ = 'pratyush'
import requests
import json
import os
import modules.config as config
from time import sleep
from requests.auth import HTTPBasicAuth
from ibm_watson import DiscoveryV1
from pprint import pprint
import modules.discovery_helper as dh_help
import modules.common_helper as cm_help
import sys
import pandas as pd
import modules.config as config
import math
import re
import hashlib
import unicodedata
from bs4 import BeautifulSoup 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [32]:
nav = pd.read_excel("Naviga_Entity_Extraction_v001.xlsx", sheet_name="Naviga")
nav = pd.DataFrame(nav)

In [33]:
nav.head()

,Person_Naviga,Person_ID,Company_Naviga,Company_ID,Location_Naviga,Location_ID
0,Kobe Bryant,167424f8-ca7f-4224-a00b-a54a5a7766c7,Chicago Bulls',167424f8-ca7f-4224-a00b-a54a5a7766c7,Mount Rushmore,167424f8-ca7f-4224-a00b-a54a5a7766c7
1,LeBron James,167424f8-ca7f-4224-a00b-a54a5a7766c7,ESPN's,167424f8-ca7f-4224-a00b-a54a5a7766c7,Los Angeles,167424f8-ca7f-4224-a00b-a54a5a7766c7
2,Michael Jordan,167424f8-ca7f-4224-a00b-a54a5a7766c7,Boston Celtics,167424f8-ca7f-4224-a00b-a54a5a7766c7,Des Plaines American Legion Post 36,7185ebf3-fa27-4fdc-a9fd-2011165a8603
3,Bill Russel,167424f8-ca7f-4224-a00b-a54a5a7766c7,Houston Rockets,167424f8-ca7f-4224-a00b-a54a5a7766c7,Edward Hotel,7185ebf3-fa27-4fdc-a9fd-2011165a8603
4,Robert Horry,167424f8-ca7f-4224-a00b-a54a5a7766c7,Orlando Magic,167424f8-ca7f-4224-a00b-a54a5a7766c7,Mannheim Road,7185ebf3-fa27-4fdc-a9fd-2011165a8603


In [34]:
len(nav["Company_Naviga"])

27906

In [17]:
df1 = pd.DataFrame(nav[["Person_Naviga","Person_ID"]])

In [10]:
df1.head()

,Person_Naviga,Person_ID
0,Kobe Bryant,167424f8-ca7f-4224-a00b-a54a5a7766c7
1,LeBron James,167424f8-ca7f-4224-a00b-a54a5a7766c7
2,Michael Jordan,167424f8-ca7f-4224-a00b-a54a5a7766c7
3,Bill Russel,167424f8-ca7f-4224-a00b-a54a5a7766c7
4,Robert Horry,167424f8-ca7f-4224-a00b-a54a5a7766c7


In [18]:
df2 = df1.duplicated(subset="Person_Naviga", keep='first')

In [15]:
df2

0        False
1        False
2        False
3        False
4        False
         ...  
27901    False
27902    False
27903    False
27904    False
27905    False
Length: 27906, dtype: bool

In [19]:
df2.value_counts()

False    27730
True       176
dtype: int64

In [20]:
for i in range(27906):
    df1.loc[i,"Person_ID"] = [df1.loc[i,"Person_ID"]]

In [21]:
for i in range(27906):
    if(df2[i]==True):
        lists=[]
        flag =0
        for j in range(27906):
            if df1.loc[j,"Person_Naviga"] == df1.loc[i,"Person_Naviga"] and i!=j:
                if flag==0:
                    first = j
                    flag=1
                lists.append(df1.loc[j,"Person_ID"][0])
        df1.loc[first,"Person_ID"].extend(lists)
        

In [22]:
for i in range(27906):
    if df2[i] == True and len(df1.loc[i,"Person_ID"])==1:
        df1.loc[i,"Person_Naviga"] = 0

        

In [23]:

df1 = df1[df1['Person_Naviga'] != 0]

In [24]:
df1.head()

,Person_Naviga,Person_ID
0,Kobe Bryant,[167424f8-ca7f-4224-a00b-a54a5a7766c7]
1,LeBron James,[167424f8-ca7f-4224-a00b-a54a5a7766c7]
2,Michael Jordan,[167424f8-ca7f-4224-a00b-a54a5a7766c7]
3,Bill Russel,[167424f8-ca7f-4224-a00b-a54a5a7766c7]
4,Robert Horry,[167424f8-ca7f-4224-a00b-a54a5a7766c7]


In [28]:
len(df1["Person_ID"])

27730

In [49]:
df3 = pd.DataFrame(nav[["Company_Naviga","Company_ID"]])
df3 = df3.dropna()

In [50]:
len(df3)

15095

In [51]:
df2 = df3.duplicated(subset="Company_Naviga", keep='first')

In [52]:
for i in range(15095):
    df3.loc[i,"Company_ID"] = [df3.loc[i,"Company_ID"]]

In [53]:
for i in range(15095):
    if(df2[i]==True):
        lists=[]
        flag =0
        for j in range(15095):
            if df3.loc[j,"Company_Naviga"] == df3.loc[i,"Company_Naviga"] and i!=j:
                if flag==0:
                    first = j
                    flag=1
                lists.append(df3.loc[j,"Company_ID"])
        df3.loc[first,"Company_ID"].extend(lists)

In [54]:
for i in range(15095):
    if df2[i] == True and len(df3.loc[i,"Company_ID"])==1:
        df3.loc[i,"Company_Naviga"] = 0

In [55]:
df3 = df3[df3['Company_Naviga'] != 0]

In [56]:
len(df3)

14677

In [59]:
df4 = pd.DataFrame(nav[["Location_Naviga","Location_ID"]])
df4 = df4.dropna()
len(df4)

13979

In [60]:
df2 = df4.duplicated(subset="Location_Naviga", keep='first')

In [61]:
for i in range(13979):
    df4.loc[i,"Location_ID"] = [df4.loc[i,"Location_ID"]]

In [62]:
for i in range(13979):
    if(df2[i]==True):
        lists=[]
        flag =0
        for j in range(13979):
            if df4.loc[j,"Location_Naviga"] == df4.loc[i,"Location_Naviga"] and i!=j:
                if flag==0:
                    first = j
                    flag=1
                lists.append(df4.loc[j,"Location_ID"])
        df4.loc[first,"Location_ID"].extend(lists)

In [63]:
for i in range(13979):
    if df2[i] == True and len(df4.loc[i,"Location_ID"])==1:
        df4.loc[i,"Location_Naviga"] = 0

In [64]:
df4 = df4[df4['Location_Naviga'] != 0]

In [65]:
len(df4)

13314

In [66]:
df_doc = pd.concat([df1,df3,df4], ignore_index=True, axis=1)
# nav = nav.append(df_doc)

In [67]:
df_doc.head()

,0,1,2,3,4,5
0,Kobe Bryant,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Chicago Bulls',[167424f8-ca7f-4224-a00b-a54a5a7766c7],Mount Rushmore,[167424f8-ca7f-4224-a00b-a54a5a7766c7]
1,LeBron James,[167424f8-ca7f-4224-a00b-a54a5a7766c7],ESPN's,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Los Angeles,"[167424f8-ca7f-4224-a00b-a54a5a7766c7, [167424..."
2,Michael Jordan,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Boston Celtics,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Des Plaines American Legion Post 36,[7185ebf3-fa27-4fdc-a9fd-2011165a8603]
3,Bill Russel,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Houston Rockets,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Edward Hotel,[7185ebf3-fa27-4fdc-a9fd-2011165a8603]
4,Robert Horry,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Orlando Magic,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Mannheim Road,[7185ebf3-fa27-4fdc-a9fd-2011165a8603]


In [68]:
df_doc = df_doc.fillna("")
df_doc = df_doc.rename(columns={0: "Person_Naviga", 1: "Person_ID", 2:"Company_Naviga", 3:"Company_ID", 4:"Location_Naviga",5:"Location_ID"})

In [69]:
df_doc.head()

,Person_Naviga,Person_ID,Company_Naviga,Company_ID,Location_Naviga,Location_ID
0,Kobe Bryant,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Chicago Bulls',[167424f8-ca7f-4224-a00b-a54a5a7766c7],Mount Rushmore,[167424f8-ca7f-4224-a00b-a54a5a7766c7]
1,LeBron James,[167424f8-ca7f-4224-a00b-a54a5a7766c7],ESPN's,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Los Angeles,"[167424f8-ca7f-4224-a00b-a54a5a7766c7, [167424..."
2,Michael Jordan,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Boston Celtics,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Des Plaines American Legion Post 36,[7185ebf3-fa27-4fdc-a9fd-2011165a8603]
3,Bill Russel,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Houston Rockets,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Edward Hotel,[7185ebf3-fa27-4fdc-a9fd-2011165a8603]
4,Robert Horry,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Orlando Magic,[167424f8-ca7f-4224-a00b-a54a5a7766c7],Mannheim Road,[7185ebf3-fa27-4fdc-a9fd-2011165a8603]


In [70]:
df_doc.columns

Index(['Person_Naviga', 'Person_ID', 'Company_Naviga', 'Company_ID',
       'Location_Naviga', 'Location_ID'],
      dtype='object')

In [71]:
df_doc.to_excel("Naviga_Entity_Extraction_v002.xlsx", sheet_name="Naviga",index=False)